In [1]:
import psycopg2
import pandas as pd
import re

In [4]:
# Crear database en psql usando commando:
# psql -U postgres
# CREATE DATABASE proyecto2;

# Conectar a la base de datos y actualizar password
conn = psycopg2.connect(
    host="localhost",
    dbname="p2", 
    user="postgres",
    password="1234",
    port="5432"
)

# definir una funcion para ejecutar consultas
def ejecutar_consulta(sql_str, select = False):
    try:
        # Crear un cursor para ejecutar las consultas
        cur = conn.cursor()
        # Ejecutar la consulta
        result = cur.execute(sql_str)   
        # Aplicar commit si la consulta es INSERT, UPDATE o DELETE     
        conn.commit()        
        # Obtener los resultados
        if select:
            rows = cur.fetchall()
            df = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])
            return df
        else:
            return result
    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()  # Revertir si hay un error
    finally:
        cur.close()

In [ ]:
sql_create_table = """
CREATE TABLE IF NOT EXISTS manga(
    title text,
    description text, 
    rating numeric,
    year numeric,
    tags text, 
    cover text, 
    merge text
);
"""

#  se importo data del csv.
#  \copy manga from '</path/to/file/filename.csv>' delimiter ',' CSV HEADER;

In [ ]:
# Prueba y verificacion de los datos
sql_str = "select * from manga;"
ejecutar_consulta(sql_str, select=True)

In [ ]:
# Probando to_tsvector
sql_str = "select to_tsvector('english', merge) as merge_vector from manga;"
ejecutar_consulta(sql_str, select=True)

In [ ]:
# Agregar columna vectorizada a la tabla
sql_str = "alter table manga add column merge_vector tsvector;"
ejecutar_consulta(sql_str, select=True)

In [ ]:
# Actualizar columna
sql_str = "update manga set merge_vector = to_tsvector('english', merge)"
ejecutar_consulta(sql_str, select=True)

In [ ]:
# Probando columna con texto vectorizado
sql_str = "select merge_vector from manga;"
ejecutar_consulta(sql_str, select=True)

In [ ]:
# Agregar un indice invertido (GIN)
sql_str = "create index manga_merge_index on manga using GIN(merge_vector)"
ejecutar_consulta(sql_str, select=True)

In [ ]:
Ns = [pow(2, i) * 1000 for i in [1, 2, 3, 4, 5, 6, 7]]

for N in Ns:

    sql_str = f""" 

    create table manga_vector_subset (
    merge_vector tsvector
    );

    insert into manga_vector_subset (merge_vector)
    select merge_vector
    from manga
    limit {N};

    create index manga_vector_subset_index
    on manga_vector_subset
    using GIN(merge_vector);

    explain analyze
    select merge_vector, ts_rank_cd(merge_vector, query_w) as rank
    from manga_vector_subset, to_tsquery('Tanjirou | Kimetsu | Nezuko' )
    query_w
    where query_w @@ merge_vector
    order by rank desc limit 10;
    """

    result = ejecutar_consulta(sql_str, True)
    execution_time = float(re.search(r'Execution Time: (\d+\.\d+)', str(result)).group(1))
    print(f"{N}: {execution_time} ms")

    sql_drop = "drop table manga_vector_subset;"
    ejecutar_consulta(sql_drop)



2000: 0.05 ms
4000: 0.046 ms
8000: 0.051 ms
16000: 0.047 ms
32000: 0.098 ms
64000: 0.155 ms
128000: 0.114 ms
